In [3]:
!pip install psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.3 MB/s eta 0:00:0000:0100:010m


In [9]:
import psycopg2

# List of names
# names = ["name1", "name2", "name3"]
names = ["bibek", "prajwal", "utsab", "abhash", "sadikshya", "manoj"]

# Connect to PostgreSQL
conn = psycopg2.connect(
      dbname=f"postgres",
      user="postgres",
      password="postgres",
      host="dlytica-academy-server-l2a-master.eastus.cloudapp.azure.com",
      port="30100"
    )


In [10]:
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [11]:
names = ["bibek", "prajwal", "utsab", "abhash", "sadikshya", "manoj"]
# Iterate over the list of names
for name in names:
    conn = psycopg2.connect(
      dbname=f"postgres",
      user="postgres",
      password="postgres",
      host="dlytica-academy-server-l2a-master.eastus.cloudapp.azure.com",
      port="30100"
    )
    # Set the isolation level to autocommit
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    # Create a cursor for the template1 connection
    cur_template = conn.cursor()

    # Execute the CREATE DATABASE statement on template1
    cur_template.execute(f"CREATE DATABASE {name}_db;")

    # Close the cursor and connection to template1
    cur_template.close()
    conn.close()

    # Connect to the newly created database
    conn_db = psycopg2.connect(
        dbname=f"{name}_db",
        user="postgres",
        password="postgres",
        host="dlytica-academy-server-l2a-master.eastus.cloudapp.azure.com",
        port="30100"
    )

    # Create a cursor for the new database connection
    cur_db = conn_db.cursor()

    # Execute additional commands (e.g., CREATE USER, ALTER DATABASE) on the new database
    cur_db.execute(f"SELECT 1 FROM pg_roles WHERE rolname='{name}';")
    if not cur_db.fetchone():
        # Create the user if it does not exist
        cur_db.execute(f"CREATE USER {name} WITH PASSWORD '{name}_123';")
        
    cur_db.execute(f"ALTER DATABASE {name}_db OWNER TO {name};")

    # Commit changes and close the cursor and connection to the new database
    conn_db.commit()
    cur_db.close()
    conn_db.close()

# Close the connection to the PostgreSQL server
conn.close()


In [31]:
# Set the isolation level to autocommit
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Create a cursor for the template1 connection
cur_template = conn.cursor()

for name in names:
   
    cur_template.execute(f"""
    SELECT pg_terminate_backend(pg_stat_activity.pid)
    FROM pg_stat_activity
    WHERE pg_stat_activity.datname = '{name}_db'
      AND pid <> pg_backend_pid();
    """)
    # Drop the database
    cur_template.execute(f"DROP DATABASE IF EXISTS {name}_db;")
    # cur_template.execute(f"DROP DATABASE IF EXISTS {name}_db;")
    